In [15]:
# Import dependencies
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from splinter import Browser

In [16]:
# URLs for use
urls = {
    'news':'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest',
    'jpl_img':'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars',
    'weather':'https://twitter.com/marswxreport?lang=en',
    'facts':'https://space-facts.com/mars/',
    'hemi':'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
}

# Ready to recieve soups
soup = {}

In [17]:
# Get browser ready
browser = Browser("chrome", headless=True)

# Navigates to URL and calls for soup to be made
def make_soup(url):
    browser.visit(url)
    
    if url == urls['jpl_img']:
        browser.click_link_by_id('full_image')
    
    soup = souper()
    return soup
 
# Makes soup
def souper():
    time.sleep(1)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    return soup

# Retrieves hemisphere image URLs
def hemi_imgs(url_list):
    full_res_urls = [] 
    
    for item in url_list:
        blank_dict = {}
        browser.visit(item['img_url'])
        new_soup = souper()
        partial_url = new_soup.find('img', class_='wide-image')['src']
        full_url = 'https://astrogeology.usgs.gov' + partial_url
        blank_dict['title'] = item['title']
        blank_dict['img_url'] = full_url
        full_res_urls.append(blank_dict)
        
    return full_res_urls

In [18]:
# Get soup objects from URLs
for url_name, url in urls.items():
    soup[url_name] = make_soup(url)
print(len(soup.items()))

5


In [19]:
# Get latest News Title and Paragraph Text 
news_dict = {}
news_dict['news_title'] = soup['news'].find('div',class_="content_title").get_text()
news_dict['news_text'] = soup['news'].find('div',class_="article_teaser_body").get_text()
print(news_dict)

{'news_title': 'NASA Social Media and Websites Win Webby Awards ', 'news_text': 'NASA\'s social media presence, the InSight mission social media accounts, NASA.gov and SolarSystem.NASA.gov will be honored at the 2019 Webby Awards - "the Oscars of the Internet."'}


In [20]:
# Get featured image URL
partial_image_url = soup['jpl_img'].find('img', class_='fancybox-image')['src']
featured_image_url = 'https://www.jpl.nasa.gov' + partial_image_url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14944_ip.jpg


In [21]:
# Get latest Mars Weather data from latest post
mars_weather = soup['weather'].find('p', class_='tweet-text').get_text()
print(mars_weather)

InSight sol 149 (2019-04-28) low -98.8ºC (-145.8ºF) high -19.5ºC (-3.0ºF)
winds from the SW at 4.6 m/s (10.2 mph) gusting to 14.2 m/s (31.7 mph)
pressure at 7.40 hPapic.twitter.com/IaUxdVEgb7


In [44]:
# Get HTML table string from site
facts_df = pd.read_html('https://space-facts.com/mars/')[0]
facts_df.columns = ['Description', 'Value']
facts_table_html = facts_df.to_html(index=False)
print(facts_table_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [39]:
# Get title and URL data for links to pages with full res images
hemi_url_list = []

head_tags = soup['hemi'].find_all('h3')

title_texts = []
for tag in head_tags:
    title_texts.append(tag.get_text())

# Handle potential 404 error from target site
try:
    # Loop through URLs for each full res image
    for x in range(4):
        blank_dict = {}
        partial_hemi_url = head_tags[x].find_parent('a')['href']
        full_hemi_url = 'https://astrogeology.usgs.gov' + partial_hemi_url
        blank_dict['title'] = title_texts[x]
        blank_dict['img_url'] = full_hemi_url
        hemi_url_list.append(blank_dict)

    # Call for full res image URLs
    hemisphere_image_urls = hemi_imgs(hemi_url_list)
    print(hemisphere_image_urls)

except IndexError:
    print("No links to full resolution images were found. Most likely 404 Error on target website.")
    hemisphere_image_urls = {}

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


In [45]:
# Create final dictionary
scraped_data = {
    'news': news_dict,
    'jpl_img': featured_image_url,
    'weather': mars_weather,
    'facts': facts_table_html,
    'hemi': hemisphere_image_urls
}
print(scraped_data)

{'news': {'news_title': 'NASA Social Media and Websites Win Webby Awards ', 'news_text': 'NASA\'s social media presence, the InSight mission social media accounts, NASA.gov and SolarSystem.NASA.gov will be honored at the 2019 Webby Awards - "the Oscars of the Internet."'}, 'jpl_img': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14944_ip.jpg', 'weather': 'InSight sol 149 (2019-04-28) low -98.8ºC (-145.8ºF) high -19.5ºC (-3.0ºF)\nwinds from the SW at 4.6 m/s (10.2 mph) gusting to 14.2 m/s (31.7 mph)\npressure at 7.40 hPapic.twitter.com/IaUxdVEgb7', 'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n   